In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.3 MB/s eta 0:00:00


In [3]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_text as text
import tensorflow_datasets as tfds

In [4]:
data = pd.read_csv('/content/gdrive/MyDrive/tam_eng/combined_data.csv')

In [5]:
data

,English,Tamil
0,"moreover all the vessels , which king ahaz in ...",ராஜாவாகிய ஆகாஸ் அரசாளும்போது தம்முடைய பாதகத்தி...
1,similar conditions will be imposed if the sri ...,சர்வதேச நாணய நிதியம் இலங்கைக்கு கடன் வழங்கினால...
2,now kornelius argues the opposite instead of e...,தற்போது அதற்கு எதிராக வாதாடுகிறார் சர்வதேச சட...
3,chrysler the third largest us automaker filed ...,அமெரிக்காவின் மூன்றாம் பெரிய கார் தயாரிப்பு நி...
4,"moreover , khan has been in exile in iran for ...",மேலும் இனைவிட்டு தலிபானால் வெளியேற்றப்பட்ட 199...
...,...,...
289446,attacking the ltte for holding onto a separati...,பிரிவினைவாத நிகழ்ச்சி நிரலைக் கொண்டுள்ளதற்காக...
289447,among the bushes they brayed; under the nettle...,"செடிகளுக்குள்ளிருந்து கதறி, காஞ்செறிகளின்கீழ் ..."
289448,"then shall you be clear from this my oath, whe...","நீ என் இனத்தாரிடத்திற்குப் போனால், என் ஆணைக்கு..."
289449,"five thousand striking employees in dongguan, ...",அக்டோபர் 6-ல் பிரதான தொழில் உற்பத்தி நகரமான ...


In [6]:
features = data['Tamil'].values
labels = data['English'].values
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [7]:
for ta, en in dataset.take(1):
  tamil = ta.numpy().decode('utf-8')
  eng = en.numpy().decode('utf-8')
  print(tamil)
  print(eng)

ராஜாவாகிய ஆகாஸ் அரசாளும்போது தம்முடைய பாதகத்தினால் எறிந்துபோட்ட சகல பணிமுட்டுகளையும் முஸ்திப்பாக்கிப் பரிசுத்தம்பண்ணினோம்; இதோ , அவைகள் கர்த்தரின் ஆலயத்திற்கு முன்பாக இருக்கிறது என்றார்கள் .
moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , and , behold , they are before the altar of the lord .


In [ ]:
#examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
#train_examples, val_examples = examples['train'], examples['validation']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/51785 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1193 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteXAHEWZ/ted_hrlr_translate-vali…

Generating test examples...:   0%|          | 0/1803 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteXAHEWZ/ted_hrlr_translate-test…

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [8]:
train_en = dataset.map(lambda ta, en: en)
train_ta = dataset.map(lambda ta, en: ta)

In [10]:
print(next(iter(train_en)))

tf.Tensor(b'moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , and , behold , they are before the altar of the lord .', shape=(), dtype=string)


In [11]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [12]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [13]:
%%time
ta_vocab = bert_vocab.bert_vocab_from_dataset(
    train_ta.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 38min 30s, sys: 19.1 s, total: 38min 49s
Wall time: 39min 19s


In [14]:
print(ta_vocab[:10])
print(ta_vocab[100:110])
print(ta_vocab[1000:1010])
print(ta_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '$', '%', '&']
['ல', 'ள', 'ழ', 'வ', 'ஷ', 'ஸ', 'ஹ', 'ா', 'ி', 'ு']
['அகடோபர', 'வி', 'இடையில', 'ராஜாவாகிய', '##ணடு', 'வெளியுறவு', '##n', 'அநதக', 'தொழில', '##ிககும']
['##ஊ', '##எ', '##ஒ', '##ஓ', '##ெ', '##ௗ', '##‘', '##’', '##“', '##”']


In [15]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)


In [16]:
write_vocab_file('/content/gdrive/MyDrive/tam_eng/ta_vocab.txt', ta_vocab)

In [17]:
%%time
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_en.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 4min 59s, sys: 1.6 s, total: 5min 1s
Wall time: 4min 57s


In [18]:
write_vocab_file('/content/gdrive/MyDrive/tam_eng/en_vocab.txt', en_vocab)

In [19]:
ta_tokenizer = text.BertTokenizer('/content/gdrive/MyDrive/tam_eng/ta_vocab.txt', **bert_tokenizer_params)
en_tokenizer = text.BertTokenizer('/content/gdrive/MyDrive/tam_eng/en_vocab.txt', **bert_tokenizer_params)

In [36]:
for ta_examples, en_examples in dataset.batch(3).take(1):
  for ex in en_examples:
    print(ex.numpy())

b'moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , and , behold , they are before the altar of the lord .'
b'similar conditions will be imposed if the sri lankan government is given an imf loan .'
b'now kornelius argues the opposite instead of enforcing the adherence to international law , new laws would now have to be devised and established .'


In [37]:
# Tokenize the examples -> (batch, word, word-piece)
token_batch = en_tokenizer.tokenize(en_examples)
# Merge the word and word-piece axes -> (batch, tokens)
token_batch = token_batch.merge_dims(-2,-1)

for ex in token_batch.to_list():
  print(ex)

[1090, 116, 83, 2820, 15, 115, 260, 41, 7776, 87, 104, 2854, 211, 763, 364, 87, 104, 60, 1832, 92, 7246, 15, 102, 126, 970, 85, 7435, 15, 85, 15, 439, 15, 108, 106, 195, 83, 1508, 84, 83, 141, 17]
[899, 395, 111, 100, 1497, 159, 83, 259, 530, 119, 89, 324, 110, 1465, 3097, 17]
[157, 51, 5095, 852, 3844, 4564, 92, 83, 2646, 816, 84, 4592, 829, 588, 1702, 83, 41, 153, 5089, 1143, 86, 192, 367, 15, 162, 1428, 140, 157, 102, 86, 100, 1226, 4409, 85, 1034, 17]


In [38]:
words = en_tokenizer.detokenize(token_batch)
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , and , behold , they are before the altar of the lord .',
       b'similar conditions will be imposed if the sri lankan government is given an imf loan .',
       b'now kornelius argues the opposite instead of enforcing the adherence to international law , new laws would now have to be devised and established .'],
      dtype=object)>

In [39]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

In [40]:
words = en_tokenizer.detokenize(add_start_end(token_batch))
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'[START] moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , and , behold , they are before the altar of the lord . [END]',
       b'[START] similar conditions will be imposed if the sri lankan government is given an imf loan . [END]',
       b'[START] now kornelius argues the opposite instead of enforcing the adherence to international law , new laws would now have to be devised and established . [END]'],
      dtype=object)>

In [41]:
def cleanup_text(reserved_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)

  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

In [42]:
en_examples.numpy()

array([b'moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , and , behold , they are before the altar of the lord .',
       b'similar conditions will be imposed if the sri lankan government is given an imf loan .',
       b'now kornelius argues the opposite instead of enforcing the adherence to international law , new laws would now have to be devised and established .'],
      dtype=object)

In [43]:
token_batch = en_tokenizer.tokenize(en_examples).merge_dims(-2,-1)
words = en_tokenizer.detokenize(token_batch)
words

<tf.RaggedTensor [[b'moreover', b'all', b'the', b'vessels', b',', b'which', b'king', b'ahaz',
  b'in', b'his', b'reign', b'did', b'cast', b'away', b'in', b'his',
  b'transgression', b',', b'have', b'we', b'prepared', b'and',
  b'sanctified', b',', b'and', b',', b'behold', b',', b'they', b'are',
  b'before', b'the', b'altar', b'of', b'the', b'lord', b'.']                ,
 [b'similar', b'conditions', b'will', b'be', b'imposed', b'if', b'the',
  b'sri', b'lankan', b'government', b'is', b'given', b'an', b'imf', b'loan',
  b'.']                                                                     ,
 [b'now', b'kornelius', b'argues', b'the', b'opposite', b'instead', b'of',
  b'enforcing', b'the', b'adherence', b'to', b'international', b'law', b',',
  b'new', b'laws', b'would', b'now', b'have', b'to', b'be', b'devised',
  b'and', b'established', b'.']                                             ]>

In [44]:
cleanup_text(reserved_tokens, words).numpy()

array([b'moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , and , behold , they are before the altar of the lord .',
       b'similar conditions will be imposed if the sri lankan government is given an imf loan .',
       b'now kornelius argues the opposite instead of enforcing the adherence to international law , new laws would now have to be devised and established .'],
      dtype=object)

In [45]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:

    # Include a tokenize signature for a batch of strings.
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)

In [46]:
tokenizers = tf.Module()
tokenizers.en = CustomTokenizer(reserved_tokens, '/content/gdrive/MyDrive/tam_eng/en_vocab.txt')
tokenizers.ta = CustomTokenizer(reserved_tokens, '/content/gdrive/MyDrive/tam_eng/ta_vocab.txt')

In [47]:
model_name = '/content/gdrive/MyDrive/tam_eng/ted_hrlr_translate_ta_en_converter'
tf.saved_model.save(tokenizers, model_name)

In [48]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.ta.get_vocab_size().numpy()

7750

In [49]:
tokens = reloaded_tokenizers.en.tokenize(['Hello where are you i am going to park!'])
tokens.numpy()

array([[   2, 3157,  299,  253,  106,  101,   49,  349,  434,   86, 4784,
           4,    3]])

In [50]:
text_tokens = reloaded_tokenizers.en.lookup(tokens)
text_tokens

<tf.RaggedTensor [[b'[START]', b'hell', b'##o', b'where', b'are', b'you', b'i', b'am',
  b'going', b'to', b'park', b'!', b'[END]']]>

In [51]:
round_trip = reloaded_tokenizers.en.detokenize(tokens)

print(round_trip.numpy()[0].decode('utf-8'))

hello where are you i am going to park !
